In [2]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("../data/nuanz.txt", encoding="utf-8")
text_documents = loader.load()
text_documents

[Document(metadata={'source': '../data/nuanz.txt'}, page_content='Nuanz is build from KPR fintech')]

In [3]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_paths=("https://www.nseindia.com/market-data/live-equity-market",),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("tab-content","py-1"))),
)
web_documents = loader.load()
web_documents

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://www.nseindia.com/market-data/live-equity-market'}, page_content='\n\n\n\n\n\n                      Equity/Stock\n                    \n\n                      T0 settlement\n                    \n\n                      SME Market\n                    \n\n                      Sovereign Gold Bonds (SGB)\n                    \n\n                      Exchange Traded Funds\n                    \n\n                      Block Deals\n                    \n\n\n\n\n\n\n\n\nCategory 1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                Advances - \n\n\n                                Declines - \n\n\n                                Unchanged - \n\n\n\n\n\n\n\n\n Download (.csv)\n\n\n\n\n\n\n\n\n\n\nChange denomination\n\n\n\nLakhs\n\n\n\nCrores\n\n\n\nBillions\n\n\n\n\n\n\nStreaming Off\n\n\nOn\n\n\n\n\n1\n\n\nof 80\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                            Advances\n        

In [4]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("../data/sample.pdf")
pdf_documents = loader.load()
pdf_documents

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2019', 'creator': 'Microsoft® PowerPoint® 2019', 'creationdate': '2025-04-25T19:41:13+05:30', 'title': 'Reliance Industries', 'author': 'Shashank11 Singh', 'moddate': '2025-04-25T19:41:13+05:30', 'source': '../data/sample.pdf', 'total_pages': 73, 'page': 0, 'page_label': '1'}, page_content='1\nFinancial Results Presentation \nFY 2024-25 / 4Q FY2025\n25 April 2025'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2019', 'creator': 'Microsoft® PowerPoint® 2019', 'creationdate': '2025-04-25T19:41:13+05:30', 'title': 'Reliance Industries', 'author': 'Shashank11 Singh', 'moddate': '2025-04-25T19:41:13+05:30', 'source': '../data/sample.pdf', 'total_pages': 73, 'page': 1, 'page_label': '2'}, page_content='2\nForward Looking Statement\nThis presentation contains forward-looking statements which may be identified by their use of words like “plans,” \n“expects,” “will,” “anticipates,” “believes,” “intends,” “projects,” “estimates” 

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(text_documents + web_documents + pdf_documents)
texts[0:5]

[Document(metadata={'source': '../data/nuanz.txt'}, page_content='Nuanz is build from KPR fintech'),
 Document(metadata={'source': 'https://www.nseindia.com/market-data/live-equity-market'}, page_content='Equity/Stock\n                    \n\n                      T0 settlement\n                    \n\n                      SME Market\n                    \n\n                      Sovereign Gold Bonds (SGB)\n                    \n\n                      Exchange Traded Funds\n                    \n\n                      Block Deals\n                    \n\n\n\n\n\n\n\n\nCategory 1\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                Advances - \n\n\n                                Declines - \n\n\n                                Unchanged - \n\n\n\n\n\n\n\n\n Download (.csv)\n\n\n\n\n\n\n\n\n\n\nChange denomination\n\n\n\nLakhs\n\n\n\nCrores\n\n\n\nBillions\n\n\n\n\n\n\nStreaming Off\n\n\nOn\n\n\n\n\n1\n\n\nof 80\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [6]:
## vector embedding and vector store
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(texts[:10], embedding=OllamaEmbeddings(model="nomic-embed-text"))



C:\Users\Guru\AppData\Local\Temp\ipykernel_12480\1865113698.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db = Chroma.from_documents(texts[:10], embedding=OllamaEmbeddings(model="nomic-embed-text"))


In [7]:
query = "What is Nuance Communications?"
docs = db.similarity_search(query)
docs[0].page_content  


'Nuanz is build from KPR fintech'

In [8]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
db1 = FAISS.from_documents(texts[:10], embedding=OllamaEmbeddings(model="nomic-embed-text"))



In [9]:
query = "What is Nuance Communications?"
docs = db1.similarity_search(query)
docs[0].page_content  

'Nuanz is build from KPR fintech'

In [10]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2:1b", temperature=0.7)


In [11]:
## Design chatprompt template
from langchain.prompts import ChatPromptTemplate
prompts = ChatPromptTemplate.from_template(
    "Answer the question based on the context below. \n\n.\nContext: {context}\n\nQuestion: {input}\nAnswer:")


In [12]:
## Chain for QA
from langchain.chains.combine_documents import create_stuff_documents_chain
chain = create_stuff_documents_chain(llm, prompt=prompts)

In [13]:
retriever = db1.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D4597DD2B0>, search_kwargs={})

In [14]:
from langchain.chains import create_retrieval_chain
qa_chain = create_retrieval_chain(retriever, chain)

In [17]:
response = qa_chain.invoke({"input": "What is Nuance Communications?"})
response["answer"]


KeyboardInterrupt



In [18]:
response = qa_chain.invoke({"input": "Can you give me the 1-year sharpe ratio of nifty50"})
response["answer"]

"Based on the provided context, I'll calculate the 1-year Sharpe Ratio for NIFTY 500 (Nifty 50).\n\n**Data Needed:**\n\n* Historical closing prices of NIFTY 500 for the year ending March 2023.\n* Historical returns of NIFTY 500 for the same period.\n\n**Calculations:**\n\nAssuming historical data for March 2022 to February 2023, I'll use the following assumptions:\n\n* Average annual return of NIFTY 500 = -1.5% (Note: This is an estimate and actual returns may vary)\n* Sharpe Ratio formula: SR = (Expected Return - Risk-Free Rate) / Standard Deviation\n\n**Historical Data:**\n\n| Date | Closing Price | Return |\n| --- | --- | --- |\n| Mar 2022 | 6,144.00 | 0% |\n| Feb 2023 | 6,300.00 | -1.5% |\n\nUsing a financial calculator or software (e.g., Excel), I calculated the expected return and standard deviation of NIFTY 500 for the year:\n\n* Expected Return = -1.5%/year ≈ -13.33%\n* Standard Deviation ≈ ±21.63%\n\n**Sharpe Ratio Calculation:**\n\nSR = (-13.33% - 0%) / 21.63% ≈ -0.61\n\nThe 